In [1]:
%pip install --upgrade --quiet  langchain-google-genai pillow

Note: you may need to restart the kernel to use updated packages.


In [3]:

import os
os.environ['GOOGLE_API_KEY'] = os.environ.get('GOOGLE_API_KEY')

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("Write a ballad about LangChain")
print(result.content)



In realms of code, where knowledge flows,
There lived a marvel, LangChain's throes.
A model vast, with wisdom deep,
In language's embrace, its secrets kept.

From text to text, it wove its thread,
A tapestry of words, where meaning spread.
With every query, it would delve,
Unveiling insights, stories to tell.

Its syntax smooth, a graceful dance,
It parsed and understood, with keenest glance.
From sentiment to tone, it could discern,
The hidden nuances that words would turn.

In conversations, it shone so bright,
A virtual sage, with boundless light.
It answered questions, wise and true,
Guiding users through the labyrinth of view.

But with great power comes a weighty task,
To use its knowledge, not as a mask.
For in the hands of those who sought to deceive,
It could be twisted, its purpose to leave.

So let us wield LangChain with care,
 its potential, both great and fair.
May it be a beacon of knowledge and truth,
Guiding us toward a world imbued with proof.

In the annals of AI, it

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain_community.utilities.sql_database import SQLDatabase

db_user = os.environ.get('DB_USER')
db_password = os.environ.get('DB_PASSWORD')
db_host = os.environ.get('DB_HOST')
db_name = os.environ.get('DB_NAME')



# db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=1,include_tables=['customers','orders'],custom_table_info={'customers':"customer"})
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)


mysql
['adminapp_companyprofile', 'adminapp_jobapplication', 'adminapp_jobposting', 'auth_group', 'auth_group_permissions', 'auth_permission', 'auth_user', 'auth_user_groups', 'auth_user_user_permissions', 'django_admin_log', 'django_content_type', 'django_migrations', 'django_session', 'userapp_candidatedetails']

CREATE TABLE adminapp_companyprofile (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	company_name VARCHAR(255) NOT NULL, 
	company_details LONGTEXT NOT NULL, 
	logo VARCHAR(100) NOT NULL, 
	company_website VARCHAR(200) NOT NULL, 
	user_id INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	CONSTRAINT adminapp_companyprofile_user_id_c07b6444_fk_auth_user_id FOREIGN KEY(user_id) REFERENCES auth_user (id)
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from adminapp_companyprofile table:
id	company_name	company_details	logo	company_website	user_id
1	Botit	Robotics Company	company_logos/tf_logo_social.png	https://www.tensorflow.org/	1
2	Google	Google LLC is an American 

In [12]:
load_dotenv()

True

In [13]:


project_name = 'nl2sql test with gemini'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {project_name}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.environ.get('LANGCHAIN_API_KEY')


In [14]:
from langchain.chains import create_sql_query_chain

generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "How many jobs are posted ?"})
# "How many candidates applied for job ?"
print(query)


SELECT COUNT(*) AS `Number of Jobs`
FROM adminapp_jobposting;


In [15]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)


'[(17,)]'

In [17]:
query = generate_query.invoke({"question": "how many are applied for job with job_id="})
print(query)

SELECT COUNT(*) 
FROM adminapp_jobapplication
WHERE job_id = '23';


In [18]:
query = generate_query.invoke({"question": "Shortlist top 3 candidates who applied for job with job_id=23"})
print(query)

```sql
SELECT
  candidate_name,
  score,
  experience,
  age
FROM adminapp_jobapplication
WHERE
  job_id = 23
ORDER BY
  score DESC
LIMIT 3;
```


## With SQL Agent

In [20]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, verbose=True)

In [21]:
agent_executor.invoke(
    {
        "input": "Shortlist top 3 candidates who applied for job with job_id=17"
    }
)



> Entering new SQL Agent Executor chain...
I should first check in the database which tables exist.
Action: sql_db_list_tables
Action Input: adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetailsAction: sql_db_schema
Action Input: adminapp_jobapplication
CREATE TABLE adminapp_jobapplication (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	job_id VARCHAR(50) NOT NULL, 
	job_title VARCHAR(100) NOT NULL, 
	candidate_name VARCHAR(100) NOT NULL, 
	score DECIMAL(5, 2) NOT NULL, 
	phone VARCHAR(15) NOT NULL, 
	email VARCHAR(254) NOT NULL, 
	experience INTEGER UNSIGNED NOT NULL, 
	age INTEGER UNSIGNED NOT NULL, 
	resume_link VARCHAR(200) NOT NULL, 
	skills JSON NOT NULL, 
	technologies JSON NOT NULL, 
	score_summary LONGTEXT, 
	PRIMARY KEY (id), 
	CONSTRAINT adminapp_jobapplicati

{'input': 'Shortlist top 3 candidates who applied for job with job_id=17',
 'output': 'Mohammed Shaneeb, Zaheer K Z, ALEXANDER OWENS'}

In [26]:
result = agent_executor.invoke(
    {
        "input": "Shortlist top 3 candidates who applied for job with job_id=17.I want the candidate_name column,score,email,resume_lik and skills .give me output in tabular format"
    }
)



> Entering new SQL Agent Executor chain...
Action: sql_db_query
Action Input: ```sql
SELECT candidate_name,score,email,resume_lik,skills
FROM candidate
WHERE job_id=17
ORDER BY score DESC
LIMIT 3;
```
Action Input: ```sql
SELECT * FROM candidate;
```Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT candidate_name,score,email,resume_lik,skills\nFROM candidate\nWHERE ' at line 1")
[SQL: ```sql
SELECT candidate_name,score,email,resume_lik,skills
FROM candidate
WHERE job_id=17
ORDER BY score DESC
LIMIT 3;
```
Action Input: ```sql
SELECT * FROM candidate;
```]
(Background on this error at: https://sqlalche.me/e/20/f405)Action: sql_db_query_checker
Action Input: ```sql
SELECT candidate_name,score,email,resume_lik,skills
FROM candidate
WHERE job_id=17
ORDER BY score DESC
LIMIT 3;
``````sql
SELECT candidate_name,score,email,resume_lik,skills
FROM c

In [27]:
print(result['output'])

Agent stopped due to iteration limit or time limit.


In [ ]:
('from column with job_id=20,Short list top  candidates'

In [28]:
result = agent_executor.invoke(
    {
        "input": "from table adminapp_jobapplication with cloumn job_id=20.Short list top  candidates"
    }
)



> Entering new SQL Agent Executor chain...
The query should look something like:
```sql
SELECT * FROM adminapp_jobapplication
WHERE job_id = 20
ORDER BY score DESC
LIMIT 10
```
Action: sql_db_query_checker
Action Input:
```sql
SELECT * FROM adminapp_jobapplication
WHERE job_id = 20
ORDER BY score DESC
LIMIT 10
``````sql
SELECT * FROM adminapp_jobapplication
WHERE job_id = 20
ORDER BY score DESC
LIMIT 10
```I can use the query to get the top candidates
Action: sql_db_query
Action Input:
```sql
SELECT * FROM adminapp_jobapplication
WHERE job_id = 20
ORDER BY score DESC
LIMIT 10
```Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT * FROM adminapp_jobapplication\nWHERE job_id = 20\nORDER BY score DE' at line 1")
[SQL: ```sql
SELECT * FROM adminapp_jobapplication
WHERE job_id = 20
ORDER BY score DESC
LIMIT 10
```]
(Background on this error at: h

In [29]:
result

{'input': 'from table adminapp_jobapplication with cloumn job_id=20.Short list top  candidates',
 'output': 'Agent stopped due to iteration limit or time limit.'}